In [1]:
# Prerequisite for linux

!apt-get -y install openjdk-8-jdk-headless
!wget http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install findspark kaggle

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u191-b12-2ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
--2019-04-10 09:56:52--  http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.233.100, 64.50.236.52, 140.211.166.134, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.233.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230778742 (220M) [application/x-gzip]
Saving to: ‘spark-2.4.1-bin-hadoop2.7.tgz’

spark-2.4.1-bin-had 100%[===================>] 220.09M  26.6MB/s    in 16s     

2019-04-10 09:57:08 (13.9 MB/s) - ‘spark-2.4.1-bin-hadoop2.7.tgz’ saved [230778742/230778742]



In [2]:
import os

# setup environment variables
os.environ["JAVA_HOME"] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ["SPARK_HOME"] = '/content/spark-2.4.1-bin-hadoop2.7'
os.environ['KAGGLE_USERNAME'] = 'ronaldsumbayak'
os.environ['KAGGLE_KEY'] = '771009dffffa0cd0883d5fb98594d756'

# verify
print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME'])
print(os.environ['KAGGLE_USERNAME'])
print(os.environ['KAGGLE_KEY'])

/usr/lib/jvm/java-8-openjdk-amd64
/content/spark-2.4.1-bin-hadoop2.7
ronaldsumbayak
771009dffffa0cd0883d5fb98594d756


In [3]:
!kaggle datasets download -d olistbr/brazilian-ecommerce
!unzip brazilian-ecommerce.zip -d data

 78% 33.0M/42.4M [00:00<00:00, 37.0MB/s]
100% 42.4M/42.4M [00:00<00:00, 66.8MB/s]
Archive:  brazilian-ecommerce.zip
  inflating: data/olist_customers_dataset.csv  
  inflating: data/olist_sellers_dataset.csv  
  inflating: data/olist_order_reviews_dataset.csv  
  inflating: data/olist_order_payments_dataset.csv  
  inflating: data/product_category_name_translation.csv  
  inflating: data/olist_products_dataset.csv  
  inflating: data/olist_order_items_dataset.csv  
  inflating: data/olist_orders_dataset.csv  
  inflating: data/olist_geolocation_dataset.csv  


In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [0]:
orders = spark.read.csv('data/olist_orders_dataset.csv', header=True)
order_items = spark.read.csv('data/olist_order_items_dataset.csv', header=True)
products = spark.read.csv('data/olist_products_dataset.csv', header=True)

In [7]:
print('Orders      :  {} rows'.format(orders.count()))
print('Order Items : {} rows'.format(order_items.count()))
print('Products    :  {} rows'.format(products.count()))

Orders      :  99441 rows
Order Items : 112650 rows
Products    :  32951 rows


In [8]:
orders.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_carrier_date: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_estimated_delivery_date: string (nullable = true)



In [9]:
order_items.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: string (nullable = true)
 |-- freight_value: string (nullable = true)



In [10]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: string (nullable = true)
 |-- product_description_lenght: string (nullable = true)
 |-- product_photos_qty: string (nullable = true)
 |-- product_weight_g: string (nullable = true)
 |-- product_length_cm: string (nullable = true)
 |-- product_height_cm: string (nullable = true)
 |-- product_width_cm: string (nullable = true)



In [11]:
orders.show(10, truncate=False)

+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|order_id                        |customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b7cc49136f2d6af7|9ef432eb6251297304e76186b10a928d|delivered   |2017-10-02 10:56:33     |2017-10-02 11:07:15|2017-10-04 19:55:00         |2017-10-10 21:25:13          |2017-10-18 00:00:00          |
|53cdb2fc8bc7dce0b6741e2150273451|b0830fb4747a6c6d20dea0b8c802d7ef|delivered   |2018-07-24 20:41:37     |2018-07-26 03:24:27|2018-07-26 14:3

In [12]:
order_items.show(10, truncate=False)

+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+------+-------------+
|order_id                        |order_item_id|product_id                      |seller_id                       |shipping_limit_date|price |freight_value|
+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+------+-------------+
|00010242fe8c5a6d1ba2dd792cb16214|1            |4244733e06e7ecb4970a6e2683c13e61|48436dade18ac8b2bce089ec2a041202|2017-09-19 09:45:35|58.90 |13.29        |
|00018f77f2f0320c557190d7a144bdd3|1            |e5f2d52b802189ee658865ca93d83a8f|dd7ddc04e1b6c2c614352b383efe2d36|2017-05-03 11:05:13|239.90|19.93        |
|000229ec398224ef6ca0657da4fc703e|1            |c777355d18b72b67abbeef9df44fd0fd|5b51032eddd242adc84c38acab88f23d|2018-01-18 14:48:30|199.00|17.87        |
|00024acbcdf0a6daa1e931b038114c75|1            |7634da152a4610f1

In [13]:
products.show(10, truncate=False)

+--------------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id                      |product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff4541ed26657ea517e5|perfumaria           |40                 |287                       |1                 |225             |16               |10               |14              |
|3aa071139cb16b67ca9e5dea641aaa2f|artes                |44                 |276                       |1                 |1000            |30               |18               |20              |
|96bd76ec8810374ed1b65e291975717f|e

In [0]:
# Remove duplicate pair of (order_id, product_id) to avoid
# error FPGrowth algorithm can't take duplicate
# item in the same row
distinct_order_items = order_items.select('order_id', 'product_id').distinct()

In [15]:
distinct_order_items.count()

102425

In [0]:
import pyspark.sql.functions as F

In [0]:
# Group items of same order together
data = (distinct_order_items.groupby('order_id')
                            .agg(F.collect_list('product_id').alias('items')))

In [18]:
data.count()

98666

In [19]:
data.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [20]:
data.show(truncate=False)

+--------------------------------+--------------------------------------------------------------------+
|order_id                        |items                                                               |
+--------------------------------+--------------------------------------------------------------------+
|014405982914c2cde2796ddcf0b8703d|[e95ee6822b66ac6058e2e4aff656071a, 6782d593f63105318f46bbf7633279bf]|
|019886de8f385a39b75bedbb726fd4ef|[e9a69340883a438c3f91739d14d3a56d]                                  |
|01a6ad782455876aa89081449d49c452|[036734b5a58d5d4f46b0616ddc047ced]                                  |
|01d907b3e209269e120a365fc2b97524|[b1434a8f79cb3528540d9b21e686e823]                                  |
|028dc52e12ddda803ec1e35eb0b7b0d9|[d86a6c48f83b045cbba6df84926a1f25]                                  |
|036dd381dfb3ec75e0a63e14828cc871|[aa8d88eb4b9cb38894e33fa624c4287f]                                  |
|03ebfa9712b7dbc7031291856263b314|[aa6746e94490239d3d9ee6ab89779

In [0]:
from pyspark.ml.fpm import FPGrowth

In [0]:
# Seems like this dataset does not suit well with FPGrowth.
# minSupport and minConfidence needs to be lowered down to
# a really low value in order to get a proper associationRules.

fp = FPGrowth(minSupport=0.00005, minConfidence=0.00005)

In [0]:
# Fit the model
fpm = fp.fit(data)

In [24]:
fpm.freqItemsets.show(truncate=False)

+----------------------------------+----+
|items                             |freq|
+----------------------------------+----+
|[a4aa7c1427c31344e5f7cc3d839fe562]|42  |
|[632c37951cab36a4e928428cbc4fa4f4]|5   |
|[b60856ce32d90658dbf99b9485327c25]|26  |
|[33fac76e7f17367dedaef7954bbb33b4]|20  |
|[03bb06cda40712fb8473f7962fb7d198]|8   |
|[20a64c9ab614812a2634ebbd69bb4a02]|8   |
|[c18ac1d6cc053c0ec7296b994ae6ffeb]|7   |
|[71fc2a7d96a222b04e4131cfc006f265]|5   |
|[6b42b1f7dfb74bb23a01605fbedd5b85]|5   |
|[5e53af05c040b01584b81b3549945faf]|16  |
|[be1c4cf8a2d2aade623eb7a646d2a0bc]|14  |
|[f89cd865cac300a9bf1320dd8f0fa223]|12  |
|[a5e28e21acf7c99188ad33f688f18837]|7   |
|[d3b023772520cd1c6ea58f6701679d4b]|6   |
|[3a7d90845985f39048815a6939c56be4]|6   |
|[00878d953636afec00d3e85d55a12e7f]|11  |
|[2ff17002562478fb03cd44f09e7ca51a]|10  |
|[fa3bc8d73fe3b0ec061190651433a417]|6   |
|[11fb16e86df4d50cd2c289ede0b70f42]|9   |
|[a883c6cc6b971869a2c962f76fb73ab1]|5   |
+---------------------------------

In [25]:
fpm.associationRules.show(truncate=False)

+----------------------------------+----------------------------------+--------------------+------------------+
|antecedent                        |consequent                        |confidence          |lift              |
+----------------------------------+----------------------------------+--------------------+------------------+
|[3f14d740544f37ece8a9e7bc8349797e]|[36f60d45225e60c7da4558b070ce4b60]|0.14814814814814814 |131.68635301968632|
|[0aabfb375647d9738ad0f7b4ea3653b1]|[6c3effec7c8ddba466d4f03f982c7aa3]|0.036231884057971016|38.030373111316685|
|[5d790355cbeded0cd60e25cbc4c527a2]|[5fc3e6a4b52b0c414458104ed4037f1c]|0.375               |1947.3552631578946|
|[4d0ec1e9b95fb62f9a1fbe21808bf3b1]|[9ad75bd7267e5c724cb42c71ac56ca72]|0.46153846153846156 |1084.2417582417584|
|[ee57070aa3b24a06fdd0e02efd2d757d]|[0d85c435fd60b277ffb9e9b0f88f927a]|0.2608695652173913  |408.55486542443066|
|[18486698933fbb64af6c0a255f7dd64c]|[dbb67791e405873b259e4656bf971246]|0.4117647058823529  |625.03348416

In [0]:
def get_random_items(n):
    """
    This function will be used to retrieve items from
    FPGrowth model's association rules to make sure we
    get items that has prediction.
    """
    items = fpm.associationRules \
               .select('antecedent') \
               .orderBy(F.rand()) \
               .limit(n) \
               .collect()
    return list(map(lambda x: x[0], items))

In [0]:
# Crate fake orders for testing
fake_orders_n = 20
fake_orders_id = range(1, fake_orders_n+1)
fake_orders_items = get_random_items(fake_orders_n)
fake_orders = list(zip(fake_orders_id, fake_orders_items))

In [0]:
# Convert to spark DataFrame
test_data = spark.createDataFrame(fake_orders, ['order_id', 'items'])

In [29]:
test_data.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [30]:
# final test DataFrame
test_data.show(truncate=False)

+--------+----------------------------------+
|order_id|items                             |
+--------+----------------------------------+
|1       |[53759a2ecddad2bb87a079a1f1519f73]|
|2       |[e5ae72c62ebfa708624f5029d609b160]|
|3       |[f4f67ccaece962d013a4e1d7dc3a61f7]|
|4       |[368c6c730842d78016ad823897a372db]|
|5       |[53759a2ecddad2bb87a079a1f1519f73]|
|6       |[368c6c730842d78016ad823897a372db]|
|7       |[a50acd33ba7a8da8e9db65094fa990a4]|
|8       |[53759a2ecddad2bb87a079a1f1519f73]|
|9       |[98d61056e0568ba048e5d78038790e77]|
|10      |[0d85c435fd60b277ffb9e9b0f88f927a]|
|11      |[946344697156947d846d27fe0d503033]|
|12      |[422879e10f46682990de24d770e7f83d]|
|13      |[36f60d45225e60c7da4558b070ce4b60]|
|14      |[060cb19345d90064d1015407193c233d]|
|15      |[6c3effec7c8ddba466d4f03f982c7aa3]|
|16      |[389d119b48cf3043d311335e499d9c6b]|
|17      |[18486698933fbb64af6c0a255f7dd64c]|
|18      |[b7d94dc0640c7025dc8e3b46b52d8239]|
|19      |[e6b314a2236c162ede1a879

In [0]:
# Get predictions
predictions = fpm.transform(test_data)

In [32]:
# Show predictions.
# Schema of products data does not contains
# product name. So we only show it by
# product's id
predictions.show(truncate=False)

+--------+----------------------------------+------------------------------------------------------------------------------------------------------+
|order_id|items                             |prediction                                                                                            |
+--------+----------------------------------+------------------------------------------------------------------------------------------------------+
|1       |[53759a2ecddad2bb87a079a1f1519f73]|[422879e10f46682990de24d770e7f83d, 389d119b48cf3043d311335e499d9c6b, 368c6c730842d78016ad823897a372db]|
|2       |[e5ae72c62ebfa708624f5029d609b160]|[5b8a5a9417210b1b84b67b9a7aefb935]                                                                    |
|3       |[f4f67ccaece962d013a4e1d7dc3a61f7]|[4fcb3d9a5f4871e8362dfedbdb02b064]                                                                    |
|4       |[368c6c730842d78016ad823897a372db]|[389d119b48cf3043d311335e499d9c6b, 53759a2ecddad2bb87a079a1f1